In [2]:
import nltk
from collections import defaultdict

In [5]:
with open('whatsapp_chat_with_system_and_media.txt', 'r', encoding='utf-8') as f:
    content = f.readlines()

In [6]:
ppl=defaultdict(list)

for line in content:
    try:
        person = line.split(':')[2][7:]
        text = nltk.sent_tokenize(':'.join(line.split(':')[3:]))
        ppl[person].extend(text)   # If key exists (person), extend list with value (text),
                                   # if not create a new key, with value added to list
    except:
        print(line)  # in case reading a line fails, examine why
        pass

3/20/25, 9:07 AM - Alice changed the group description.

3/20/25, 10:03 AM - Messages to this group are now secured with end-to-end encryption.

3/20/25, 2:28 PM - Messages to this group are now secured with end-to-end encryption.

3/20/25, 4:16 PM - Messages to this group are now secured with end-to-end encryption.

3/20/25, 4:33 PM - Messages to this group are now secured with end-to-end encryption.

3/20/25, 2:06 PM - Alice changed this group's icon.

3/20/25, 7:56 PM - Alice changed this group's icon.

3/20/25, 6:06 PM - Messages to this group are now secured with end-to-end encryption.

3/20/25, 11:00 PM - You created group 'Weekend Vibes'.

3/21/25, 2:08 AM - Charlie left the group.

3/21/25, 3:12 AM - Alice changed the group description.

3/20/25, 11:57 PM - Charlie left the group.

3/20/25, 2:12 PM - Alice changed the group description.

3/21/25, 11:30 AM - Alice changed this group's icon.

3/21/25, 7:52 AM - You created group 'Weekend Vibes'.

3/21/25, 8:27 AM - Charlie left t

In [7]:
posts = nltk.corpus.nps_chat.xml_posts()

def extract_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

fposts = [(extract_features(p.text), p.get('class')) for p in posts]
test_size = int(len(fposts) * 0.1)
train_set, test_set = fposts[test_size:], fposts[:test_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [8]:
classifier.classify(extract_features('Hi there!'))

'Greet'

In [9]:
classifier.classify(extract_features('Do you want to watch a film later?'))

'ynQuestion'

In [12]:
import pandas as pd 
df = pd.read_csv("whatsapp_chat.csv")
df.head()

,Date,Time,Sender,Message
0,3/20/25,9:00 AM,Charlie,Oh yesss That one’s wild 🙌😎
1,3/20/25,9:03 AM,Alice,Don't forget the charger 😂
2,3/20/25,9:02 AM,Bob,Did you see the news today? 👽
3,3/20/25,9:09 AM,Alice,What time are we meeting? 🧃🧃
4,3/20/25,9:12 AM,Bob,Good luck with the interview! 🙌🙌


In [13]:
ax = df.T.plot(kind='bar', figsize=(10, 7), legend=True,
               fontsize=16, color=['y','g'])

ax.set_title("Frequency of Message Categories", fontsize= 18)
ax.set_xlabel("Message Category", fontsize=14)
ax.set_ylabel("Frequency", fontsize=14)

#plt.savefig('plots/cat_message')   # uncomment to save
plt.show()

TypeError: no numeric data to plot

In [ ]:
def extract_emojis(str):
  return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)

for key, val in ppl.items():
    emojis=extract_emojis(str(ppl[key]))
    count = Counter(emojis).most_common()[:10]

    print("{}'s emojis:\n {} \n".format(key, emojis))
    print("Most common: {}\n\n".format(count))

In [ ]:
df= pd.DataFrame(final).T  # convert dictionary to a dataframe, makes process of plotting straightforward
df.columns = ['pol', 'name', 'date', 'token']
df['pol'] = df['pol'].apply(lambda x : float(x)) # convert polarity to a float
df3 = df.groupby(['date'], as_index=False).agg('mean')
df3['name'] = 'Combined'
final =pd.concat([df2, df3])
final['date'] = pd.to_datetime(final.date, format='%d/%m/%Y') # need to chnage 'date' to a datetime object
final = final.sort_values('date')
final['x'] = final['date'].rank(method='dense', ascending=True).astype(int)
final[:6]

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(12,8))
colours=['b','y','g']
i=0

for label, df in final.groupby('name'):
    
    new=df.reset_index()
    new['rol'] = new['pol'].rolling(10).mean() # rolling mean calculation on a 10 day basis
    
    g = new.plot(x='date', y='rol', ax=ax, label=label, color=colours[i]) # rolling mean plot
    plt.scatter(df['date'].tolist(), df['pol'], color=colours[i], alpha=0.2) # underlying scatter plot
    
    i+=1

ax.set_ybound(lower=-0.1, upper=0.4)
ax.set_xlabel('Date', fontsize=15)
ax.set_ylabel('Compound Sentiment', fontsize=15)

g.set_title('10 Day Rolling Mean Sentiment', fontsize=18)

In [ ]:
pal = sns.cubehelix_palette(7, rot=-.25, light=.7)

In [ ]:
days_freq = list(df.day.value_counts().index)
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
lst = list(zip(days, pal[::-1]))
lst

In [ ]:
pal_reorder=[]

for i in days:
    #print(i)
    j=0
    for day in days_freq:
        
        if i == day:
            #print(lst[j][1])
            pal_reorder.append(lst[j][1])
        j+=1
pal_reorder   # colours ordered according to total message count for the day

In [ ]:
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
pal = sns.cubehelix_palette(7, rot=-.25, light=.7)
g = sns.FacetGrid(df[(df.float_time &gt; 8)], row="day", hue="day",   # change "day" to year_month if required
                  aspect=10, size=1.5, palette=pal_reorder, xlim=(7,24))

# Draw the densities in a few steps
g.map(sns.kdeplot, "float_time", clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2)
g.map(sns.kdeplot, "float_time", clip_on=False, color="w", lw=3, bw=.2)
g.map(plt.axhline, y=0, lw=1, clip_on=False)

# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, 0.1, label, fontweight="bold", color=color, 
            ha="left", va="center", transform=ax.transAxes, size=18)

g.map(label, "float_time")
g.set_xlabels('Time of Day', fontsize=30)
g.set_xticklabels(fontsize=20)
# Set the subplots to overlap
g.fig.subplots_adjust(hspace=-0.5)
g.fig.suptitle('Message Density by Time and Day of the Week, Shaded by Total Message Count', fontsize=22)   
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)